In [1]:
import pickle
import re
import string
import nltk
import pandas as pd
import spacy
import re
import string
from gensim import matutils, models
import scipy.sparse
from collections import Counter
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
spacy.prefer_gpu()
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [2]:
data_df = pd.read_pickle('data_clean.pickle')

In [3]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
# add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
#                   'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
# stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = TfidfVectorizer(ngram_range=(2,2))
data_cvn = cvn.fit_transform(data_df.text)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names()).T
data_dtmn.columns = data_df.index
data_dtmn

,wsb,science,ama,askreddit,worldnews,funny,dankmemes,memes,nosleep,psychology,politics,cscareerquestions,writingprompts
aa ab,0.0,0.000000,0.0,0.000123,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0
aa all,0.0,0.000138,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0
aa and,0.0,0.000358,0.0,0.000106,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0
aa as,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.00024,0.0,0.0
aa batteries,0.0,0.000000,0.0,0.000123,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
𝙄𝙏 𝘽𝙊𝘽,0.0,0.000000,0.0,0.000123,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0
𝙏𝙃𝙀 𝘽𝙐𝙄𝙇𝘿𝙀𝙍,0.0,0.000000,0.0,0.000246,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0
𝙔𝙀𝙎 𝙃𝙀,0.0,0.000000,0.0,0.000123,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0
𝚃𝚑𝚒𝚜 𝚝𝚎𝚖𝚙𝚕𝚊𝚝𝚎,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.003031,0.0,0.0,0.00000,0.0,0.0


In [4]:
# data_dtmn['index'] = data_dtmn.index
# for i, row in enumerate(data_dtmn['index']):
#     candidate = re.match('a{2,}', str(row))
#     if candidate != None:
#         data_dtmn.drop(row, inplace=True)
#     if row == 'ababababba':
#         data_dtmn.drop(row, inplace=True)
#     if row == 'wa':
#         data_dtmn.drop(row, inplace=True)       
#     if row == 'ha':
#         data_dtmn.drop(row, inplace=True)    
# data_dtmn.drop(columns=['index'], inplace=True)
# data_dtmn

In [4]:
from gensim import matutils, models
import scipy.sparse

In [5]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [ ]:
# Let's start with 2 topics
ldan = models.LdaMulticore(corpus=corpusn, num_topics=10, id2word=id2wordn, passes=50)
ldan.print_topics()

INFO - 21:53:12: using symmetric alpha at 0.1
INFO - 21:53:12: using symmetric eta at 0.1
INFO - 21:53:12: using serial LDA version on this node
INFO - 21:53:14: running online LDA training, 10 topics, 50 passes over the supplied corpus of 2001346 documents, updating every 30000 documents, evaluating every ~300000 documents, iterating 50x with a convergence threshold of 0.001000
INFO - 21:53:14: training LDA model using 15 processes
INFO - 21:53:14: PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/2001346, outstanding queue size 1
INFO - 21:53:14: PROGRESS: pass 0, dispatched chunk #1 = documents up to #4000/2001346, outstanding queue size 2
INFO - 21:53:15: PROGRESS: pass 0, dispatched chunk #2 = documents up to #6000/2001346, outstanding queue size 3
INFO - 21:53:15: PROGRESS: pass 0, dispatched chunk #3 = documents up to #8000/2001346, outstanding queue size 4
INFO - 21:53:15: PROGRESS: pass 0, dispatched chunk #4 = documents up to #10000/2001346, outstanding queue size

INFO - 21:53:39: PROGRESS: pass 0, dispatched chunk #57 = documents up to #116000/2001346, outstanding queue size 31
INFO - 21:53:41: PROGRESS: pass 0, dispatched chunk #58 = documents up to #118000/2001346, outstanding queue size 30
INFO - 21:53:41: PROGRESS: pass 0, dispatched chunk #59 = documents up to #120000/2001346, outstanding queue size 31
INFO - 21:53:42: merging changes from 30000 documents into a model of 2001346 documents
INFO - 21:53:43: topic #0 (0.100): 0.000*"aa missiles" + 0.000*"aa meetings" + 0.000*"aa all" + 0.000*"aa or" + 0.000*"aa batteries" + 0.000*"aa as" + 0.000*"aa community" + 0.000*"aa ab" + 0.000*"aa idea" + 0.000*"aa and"
INFO - 21:53:43: topic #1 (0.100): 0.000*"aa all" + 0.000*"aa as" + 0.000*"aa missiles" + 0.000*"aa batteries" + 0.000*"aa ab" + 0.000*"aa meetings" + 0.000*"aa or" + 0.000*"aa community" + 0.000*"aa funny" + 0.000*"aa idea"
INFO - 21:53:43: topic #6 (0.100): 0.000*"aa or" + 0.000*"aa all" + 0.000*"aa meetings" + 0.000*"aa missiles" + 0

INFO - 21:54:14: PROGRESS: pass 0, dispatched chunk #95 = documents up to #192000/2001346, outstanding queue size 31
INFO - 21:54:15: PROGRESS: pass 0, dispatched chunk #96 = documents up to #194000/2001346, outstanding queue size 31
INFO - 21:54:16: PROGRESS: pass 0, dispatched chunk #97 = documents up to #196000/2001346, outstanding queue size 31
INFO - 21:54:17: PROGRESS: pass 0, dispatched chunk #98 = documents up to #198000/2001346, outstanding queue size 31
INFO - 21:54:18: PROGRESS: pass 0, dispatched chunk #99 = documents up to #200000/2001346, outstanding queue size 31
INFO - 21:54:18: PROGRESS: pass 0, dispatched chunk #100 = documents up to #202000/2001346, outstanding queue size 31
INFO - 21:54:19: PROGRESS: pass 0, dispatched chunk #101 = documents up to #204000/2001346, outstanding queue size 31
INFO - 21:54:20: PROGRESS: pass 0, dispatched chunk #102 = documents up to #206000/2001346, outstanding queue size 31
INFO - 21:54:21: PROGRESS: pass 0, dispatched chunk #103 = do

INFO - 21:54:55: topic #8 (0.100): 0.000*"aa all" + 0.000*"aa ab" + 0.000*"aa meetings" + 0.000*"aa missiles" + 0.000*"aa batteries" + 0.000*"aa as" + 0.000*"aa or" + 0.000*"aa idea" + 0.000*"aa community" + 0.000*"aa and"
INFO - 21:54:55: topic diff=0.000000, rho=0.103695
INFO - 21:54:55: PROGRESS: pass 0, dispatched chunk #136 = documents up to #274000/2001346, outstanding queue size 30
INFO - 21:55:00: PROGRESS: pass 0, dispatched chunk #137 = documents up to #276000/2001346, outstanding queue size 23
INFO - 21:55:00: PROGRESS: pass 0, dispatched chunk #138 = documents up to #278000/2001346, outstanding queue size 24
INFO - 21:55:00: PROGRESS: pass 0, dispatched chunk #139 = documents up to #280000/2001346, outstanding queue size 25
INFO - 21:55:00: PROGRESS: pass 0, dispatched chunk #140 = documents up to #282000/2001346, outstanding queue size 26
INFO - 21:55:00: PROGRESS: pass 0, dispatched chunk #141 = documents up to #284000/2001346, outstanding queue size 27
INFO - 21:55:00: P

INFO - 21:55:39: merging changes from 32000 documents into a model of 2001346 documents
INFO - 21:55:41: topic #7 (0.100): 0.000*"aa meetings" + 0.000*"aa all" + 0.000*"aa missiles" + 0.000*"aa as" + 0.000*"aa or" + 0.000*"aa ab" + 0.000*"aa community" + 0.000*"aa idea" + 0.000*"aa batteries" + 0.000*"aa funny"
INFO - 21:55:41: topic #2 (0.100): 0.000*"aa all" + 0.000*"aa as" + 0.000*"aa ab" + 0.000*"aa meetings" + 0.000*"aa batteries" + 0.000*"aa or" + 0.000*"aa missiles" + 0.000*"aa community" + 0.000*"aa idea" + 0.000*"aa and"
INFO - 21:55:41: topic #4 (0.100): 0.000*"aa all" + 0.000*"aa as" + 0.000*"aa meetings" + 0.000*"aa ab" + 0.000*"aa batteries" + 0.000*"aa community" + 0.000*"aa or" + 0.000*"aa missiles" + 0.000*"aa and" + 0.000*"aa idea"
INFO - 21:55:41: topic #6 (0.100): 0.000*"aa or" + 0.000*"aa all" + 0.000*"aa meetings" + 0.000*"aa missiles" + 0.000*"aa batteries" + 0.000*"aa as" + 0.000*"aa ab" + 0.000*"aa idea" + 0.000*"aa community" + 0.000*"aa funny"
INFO - 21:55:41:

...

INFO - 22:21:46: merging changes from 30000 documents into a model of 2001346 documents
INFO - 22:21:48: topic #5 (0.100): 0.000*"aa all" + 0.000*"aa as" + 0.000*"aa missiles" + 0.000*"aa meetings" + 0.000*"aa ab" + 0.000*"aa batteries" + 0.000*"aa or" + 0.000*"aa community" + 0.000*"aa idea" + 0.000*"aa and"
INFO - 22:21:48: topic #1 (0.100): 0.000*"aa all" + 0.000*"aa as" + 0.000*"aa missiles" + 0.000*"aa batteries" + 0.000*"aa ab" + 0.000*"aa meetings" + 0.000*"aa or" + 0.000*"aa community" + 0.000*"aa idea" + 0.000*"aa funny"
INFO - 22:21:48: topic #8 (0.100): 0.000*"aa all" + 0.000*"aa ab" + 0.000*"aa meetings" + 0.000*"aa missiles" + 0.000*"aa batteries" + 0.000*"aa as" + 0.000*"aa or" + 0.000*"aa community" + 0.000*"aa idea" + 0.000*"aa and"
INFO - 22:21:48: topic #0 (0.100): 0.000*"aa missiles" + 0.000*"aa meetings" + 0.000*"aa all" + 0.000*"aa or" + 0.000*"aa as" + 0.000*"aa batteries" + 0.000*"aa ab" + 0.000*"aa community" + 0.000*"aa idea" + 0.000*"aa and"
INFO - 22:21:48: t

INFO - 22:22:21: PROGRESS: pass 1, dispatched chunk #863 = documents up to #1728000/2001346, outstanding queue size 31
INFO - 22:22:22: PROGRESS: pass 1, dispatched chunk #864 = documents up to #1730000/2001346, outstanding queue size 31
INFO - 22:22:22: PROGRESS: pass 1, dispatched chunk #865 = documents up to #1732000/2001346, outstanding queue size 31
INFO - 22:22:23: PROGRESS: pass 1, dispatched chunk #866 = documents up to #1734000/2001346, outstanding queue size 31
INFO - 22:22:24: PROGRESS: pass 1, dispatched chunk #867 = documents up to #1736000/2001346, outstanding queue size 31
INFO - 22:22:25: PROGRESS: pass 1, dispatched chunk #868 = documents up to #1738000/2001346, outstanding queue size 31
INFO - 22:22:26: PROGRESS: pass 1, dispatched chunk #869 = documents up to #1740000/2001346, outstanding queue size 31
INFO - 22:22:28: merging changes from 30000 documents into a model of 2001346 documents
INFO - 22:22:29: topic #1 (0.100): 0.000*"aa all" + 0.000*"aa as" + 0.000*"aa m

INFO - 22:23:02: PROGRESS: pass 1, dispatched chunk #901 = documents up to #1804000/2001346, outstanding queue size 25
INFO - 22:23:02: PROGRESS: pass 1, dispatched chunk #902 = documents up to #1806000/2001346, outstanding queue size 26
INFO - 22:23:02: PROGRESS: pass 1, dispatched chunk #903 = documents up to #1808000/2001346, outstanding queue size 27
INFO - 22:23:02: PROGRESS: pass 1, dispatched chunk #904 = documents up to #1810000/2001346, outstanding queue size 28
INFO - 22:23:02: PROGRESS: pass 1, dispatched chunk #905 = documents up to #1812000/2001346, outstanding queue size 29
INFO - 22:23:02: PROGRESS: pass 1, dispatched chunk #906 = documents up to #1814000/2001346, outstanding queue size 30
INFO - 22:23:02: PROGRESS: pass 1, dispatched chunk #907 = documents up to #1816000/2001346, outstanding queue size 31
INFO - 22:23:04: PROGRESS: pass 1, dispatched chunk #908 = documents up to #1818000/2001346, outstanding queue size 29
INFO - 22:23:04: PROGRESS: pass 1, dispatched ch

INFO - 22:23:41: topic #5 (0.100): 0.000*"aa all" + 0.000*"aa as" + 0.000*"aa missiles" + 0.000*"aa meetings" + 0.000*"aa ab" + 0.000*"aa batteries" + 0.000*"aa or" + 0.000*"aa community" + 0.000*"aa idea" + 0.000*"aa and"
INFO - 22:23:41: topic #3 (0.100): 0.000*"aa all" + 0.000*"aa missiles" + 0.000*"aa as" + 0.000*"aa ab" + 0.000*"aa meetings" + 0.000*"aa batteries" + 0.000*"aa and" + 0.000*"aa community" + 0.000*"aa idea" + 0.000*"aa or"
INFO - 22:23:41: topic #2 (0.100): 0.000*"aa all" + 0.000*"aa as" + 0.000*"aa ab" + 0.000*"aa meetings" + 0.000*"aa batteries" + 0.000*"aa or" + 0.000*"aa missiles" + 0.000*"aa community" + 0.000*"aa idea" + 0.000*"aa and"
INFO - 22:23:41: topic #7 (0.100): 0.000*"aa meetings" + 0.000*"aa all" + 0.000*"aa missiles" + 0.000*"aa as" + 0.000*"aa or" + 0.000*"aa ab" + 0.000*"aa community" + 0.000*"aa idea" + 0.000*"aa batteries" + 0.000*"aa funny"
INFO - 22:23:41: topic diff=0.000000, rho=0.031581
INFO - 22:23:41: PROGRESS: pass 1, dispatched chunk #94

INFO - 22:24:16: PROGRESS: pass 1, dispatched chunk #987 = documents up to #1976000/2001346, outstanding queue size 31
